In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Model
from keras.layers import Activation, Dense,Conv2D,Dropout,Activation,Flatten,MaxPooling2D
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense,
#from tensorflow.keras.utils import to_categorical
import imblearn
from imblearn.over_sampling import RandomOverSampler,SMOTE
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import librosa
from sklearn.model_selection import train_test_split



In [ ]:
  train_path='/content/sample_data/data/train/'
  test_path='/content/sample_data/data/test/'
model_path='/content/sample_data/data/audio_classification'


In [ ]:
def features_extractorcnn(file_name):
    audio, sample_rate = librosa.load(file_name) 
    mfccs_features =librosa.feature.melspectrogram(y=audio,sr=sample_rate)
    #mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_features.reshape(128,44,1)

In [ ]:
features_extractorcnn('/content/sample_data/data/train/blue.unknown.2uelmm0g.ingestion-5b9bcd5b54-gnw46.wav').shape

(128, 44, 1)

In [ ]:
x=np.array([features_extractorcnn(train_path+'/'+f) for f in listdir(train_path) if isfile(join(train_path, f))])
x_test=np.array([features_extractorcnn(test_path+'/'+f) for f in listdir(test_path) if isfile(join(test_path, f))])

In [ ]:
y=np.array([f.split('.')[0] for f in listdir(train_path) if isfile(join(train_path, f))])
y=np.array(pd.get_dummies(y))

# y_test=np.array([f.split('.')[0] for f in listdir(test_path) if isfile(join(test_path, f))])
# y_test=np.array(pd.get_dummies(y_test))

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority')
#x_over, y_over = oversample.fit_resample(x_train, y_train)
x_over, y_over = oversample.fit_resample(x_train.reshape(len(x_train),-1), y_train)
x_over=x_over.reshape(len(x_over),128,44,1)

In [ ]:
x_over.shape

(628, 128, 44, 1)

In [ ]:
y_over.shape

(628, 16)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model=keras.Sequential([
    #cnn
    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(128,44,1)),
    keras.layers.Dropout(0.4),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    keras.layers.Dropout(0.4),
    layers.MaxPooling2D((2,2)),
    
    #dense
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    keras.layers.Dropout(0.4),
    layers.Dense(16,activation='softmax')
    
])
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
model.fit(x_over,y_over, epochs=10, validation_data=(x_val, y_val),callbacks=[callback])


Epoch 1/10
20/20 [==============================] - 5s 208ms/step - loss: 13.1059 - accuracy: 0.4952 - val_loss: 2.4812 - val_accuracy: 0.4274
Epoch 2/10
20/20 [==============================] - 4s 200ms/step - loss: 2.1208 - accuracy: 0.6067 - val_loss: 1.7334 - val_accuracy: 0.5556
Epoch 3/10
20/20 [==============================] - 4s 198ms/step - loss: 1.2710 - accuracy: 0.6354 - val_loss: 1.6871 - val_accuracy: 0.6068
Epoch 4/10
20/20 [==============================] - 4s 199ms/step - loss: 1.1530 - accuracy: 0.6608 - val_loss: 1.6531 - val_accuracy: 0.5897
Epoch 5/10
20/20 [==============================] - 4s 200ms/step - loss: 0.9802 - accuracy: 0.6975 - val_loss: 1.5991 - val_accuracy: 0.5983
Epoch 6/10
20/20 [==============================] - 4s 200ms/step - loss: 0.9760 - accuracy: 0.7054 - val_loss: 1.5303 - val_accuracy: 0.6410
Epoch 7/10
20/20 [==============================] - 4s 198ms/step - loss: 0.9158 - accuracy: 0.7118 - val_loss: 1.5258 - val_accuracy: 0.6496
Epoch

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 126, 42, 32)       320       
                                                                 
 dropout_3 (Dropout)         (None, 126, 42, 32)       0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 21, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 19, 64)        18496     
                                                                 
 dropout_4 (Dropout)         (None, 61, 19, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 9, 64)        0         
 2D)                                                  

In [ ]:
model.predict(x_test)

array([[6.2049883e-05, 3.5197718e-05, 1.7528691e-04, ..., 4.9554711e-07,
        1.7602963e-05, 8.1042534e-05],
       [5.4990873e-02, 6.7727201e-02, 6.0755800e-02, ..., 8.2017392e-02,
        5.0892837e-02, 4.4385720e-02],
       [1.6132295e-04, 1.6248221e-06, 3.2255884e-05, ..., 1.4527681e-05,
        2.4808785e-05, 7.0872353e-05],
       ...,
       [2.4223438e-02, 3.4024838e-02, 5.6298014e-02, ..., 3.9756443e-02,
        3.0284712e-02, 2.7651215e-02],
       [6.5270357e-02, 6.1713066e-02, 6.2650636e-02, ..., 6.0866829e-02,
        6.3814029e-02, 5.9097815e-02],
       [1.3338888e-02, 8.6774435e-03, 2.6392631e-02, ..., 1.8769523e-02,
        2.0001961e-02, 2.1568552e-02]], dtype=float32)

In [ ]:
classes=['blue','change_color','green','high','low','noise','off','on','party','purple','red','sleep','unknown','wakeup','white','yellow']
predictions=[classes[i] for i in np.argmax(model.predict(x_test.reshape(len(x_test-1),128,44,1)),axis=1)]

In [ ]:
test_classes=[f.split('.')[0] for f in listdir(test_path) if isfile(join(test_path, f))]

In [ ]:
df=pd.DataFrame(data={'pred':predictions,'real':test_classes})

In [ ]:
df

,pred,real
0,noise,noise
1,unknown,purple
2,noise,noise
3,unknown,unknown
4,unknown,unknown
...,...,...
218,unknown,unknown
219,unknown,unknown
220,unknown,unknown
221,purple,yellow


In [ ]:
mydf=pd.DataFrame(data={'pred':predictions,'real':test_classes})

In [ ]:
mydf

,pred,real
0,red,unknown
1,low,on
2,red,unknown
3,sleep,noise
4,sleep,unknown
...,...,...
402,green,noise
403,low,red
404,on,noise
405,low,on
